In [1]:
import os
import django

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'focalflow.settings.dev')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
import pandas as pd
from django.contrib.auth import get_user_model
from datetime import datetime

# Replace 'YourDjangoAppName' with the actual name of your Django app.
User = get_user_model()

# Define the path to the Excel file.
excel_file_path = 'courses.xlsx'

# Read the Excel data into a pandas DataFrame.
df = pd.read_excel(excel_file_path)

In [3]:
df.head(1)

,id,name,code,credit,module,semester,specialty,cursus_level,cursus_level__cursus,specialty__name,module__name,module__code,cursus_level__level
0,ff810004-c9b8-4a20-92a6-590c4345a7fa,MATHEMATIQUES AGRICOLES,EAP111,3,2,1,dc1a2e1d-1a1d-4b18-8d58-7200d96e3a9f,02245578-92b8-4af0-ad86-7f62b8b146f4,e5bf3b76-cd14-495c-8668-61015a3ca41d,ENTREPRENARIAT AGROPASTORALE,MATIERES PROFESSIONNELLES,PRO,1


In [4]:
from apps.users.models import User, UserProfile
from apps.organization.models import Organization
from apps.cameis import models

In [5]:
organization = models.Organization.objects.first()

### Adding course

In [6]:
# Remove duplicates based on the 'name' column and keep the first occurrence
df2 = df.drop_duplicates(subset='name', keep='first')

In [7]:
# import random
# import datetime
# for index, row in df2.iterrows():
#     name = row['name']
#     now = datetime.datetime.utcnow()
#     Course.objects.create(
#         organization=organization, 
#         name=name,
#         # code=f"{name[:3]}{str(int(datetime.datetime.timestamp(now)))[-2:]}{random.choice([i for i in range(10)])}{random.choice([i for i in range(10)])}", 
#         )

### Adding teaching course

In [8]:
education_year = models.EducationYear.objects.first()
instructor = models.Instructor.objects.first()

In [9]:
# for index, row in df.iterrows():
#     name = row['name']
#     credit = row['credit']
#     code = row['code']
#     semester = row['semester']
#     specialty__name = row['specialty__name']
#     cursus_level__cursus_level = row['cursus_level__level']
#     module_name = row['module__name']
#     module = models.Module.objects.filter(name=module_name).first()
#     education_term = models.EducationTerm.objects.filter(position=semester).first()
#     course = models.Course.objects.filter(name=name).first()
#     education_class = models.EducationClass.objects.filter(option__option__name=specialty__name, option__path_cursus_level_specialty__path_cursus_level__level__value=cursus_level__cursus_level).first()
#     models.TeachingCourse.objects.create(organization=organization,education_year=education_year, education_class=education_class, education_term=education_term, instructor=instructor,module=module,course=course, credit=credit, code=code)

In [10]:
# models.TeachingCourse.objects.all().delete()

In [11]:
from django.db.models import Count

In [30]:
models.TeachingCourse.objects.values(           
            "organization",
            "instructor",
            "education_class",
            "course",).annotate(name=Count("course")).count()

732

In [13]:

# Group by the specified fields and count the instances in each group
grouped_courses = models.TeachingCourse.objects.values(
    # "education_term__name",
    "course__name",
    "education_class",
).annotate(course_count=Count("id"))

# Filter groups where the count of instances is greater than 1
filtered_groups = grouped_courses.filter(course_count__gt=1)

In [14]:
filtered_groups.count()

29

In [40]:
for item in filtered_groups:
    print(item["course__name"], item["education_class"])

ANGLAIS 21c964ee-b31a-4ec0-9742-f71d8d65b250
ANGLAIS 24f7f089-dfe4-41fe-86f6-df556ca99f36
ANGLAIS 65e0d7a2-f999-4d84-8f60-03f7bab3ef2f
ANGLAIS 66820c19-bd02-4759-a6d4-f1d6ec712202
ANGLAIS a120eba3-1869-430a-9778-ae1d3e850110
ANGLAIS bad746e6-4797-4163-8d69-77a79ef09cfc
ANGLAIS efdaec30-8eeb-4796-b419-efb6c91e5890
BANKING OPERATIONS AND TECHNIQUES 2d5798d0-6942-4dd1-a8af-664b06540ed1
FRANCAIS 21c964ee-b31a-4ec0-9742-f71d8d65b250
FRANCAIS 24f7f089-dfe4-41fe-86f6-df556ca99f36
FRANCAIS 65e0d7a2-f999-4d84-8f60-03f7bab3ef2f
FRANCAIS 66820c19-bd02-4759-a6d4-f1d6ec712202
FRANCAIS a120eba3-1869-430a-9778-ae1d3e850110
FRANCAIS bad746e6-4797-4163-8d69-77a79ef09cfc
FRANCAIS efdaec30-8eeb-4796-b419-efb6c91e5890
FUNCTIONAL ENGLISH 02e5fe90-b9d6-4b57-afbd-82c7ef5ef633
FUNCTIONAL ENGLISH 4dc48e41-d4f3-42bb-80cf-249192623e3d
FUNCTIONAL ENGLISH aaa0b3aa-2887-42e7-8456-1322e7fb5c92
FUNCTIONAL FRENCH 02e5fe90-b9d6-4b57-afbd-82c7ef5ef633
FUNCTIONAL FRENCH 4dc48e41-d4f3-42bb-80cf-249192623e3d
FUNCTIONAL FRE

```bash
ANGLAIS efdaec30-8eeb-4796-b419-efb6c91e5890 deleted
BANKING OPERATIONS AND TECHNIQUES 2d5798d0-6942-4dd1-a8af-664b06540ed1 deleted
FRANCAIS efdaec30-8eeb-4796-b419-efb6c91e5890 deleted
FUNCTIONAL ENGLISH aaa0b3aa-2887-42e7-8456-1322e7fb5c92 deleted
FUNCTIONAL FRENCH aaa0b3aa-2887-42e7-8456-1322e7fb5c92 deleted
GENERAL COMPUTER aaa0b3aa-2887-42e7-8456-1322e7fb5c92 deleted
INFORMATIQUE 21c964ee-b31a-4ec0-9742-f71d8d65b250 deleted
```

In [29]:
education_class = models.EducationClass.objects.filter(id="2d5798d0-6942-4dd1-a8af-664b06540ed1").first()
print(education_class)
for item in models.TeachingCourse.objects.filter(course__name="FRANCAIS", education_class=education_class):
    print(item, item.code, item.education_term)

Niveau 2-BUSINESS AND FINANCE-BANKING AND FINANCE - 1


In [27]:
models.TeachingCourse.objects.filter(code="GC208").first()

<TeachingCourse: Q1695150185 - FRANCAIS>